In [ ]:
import os
from PIL import Image

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1,2"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
from utils.frame_utils import read_gen, flow_to_image

#L_B_D = "test_gen_L_B_D/"
#L_B_U = "test_gen_L_B_U/"
#L_F_D = "test_gen_L_F_D/"
#L_F_U = "test_gen_L_F_U/"
#R_B_D = "test_gen_R_B_D/"
#R_B_U = "test_gen_R_B_U/"
R_F_D = "test_gen_R_F_D/"
#R_F_U = "test_gen_R_F_U/"

RB = "castle/test_gen_RB/"
RD = "castle/test_gen_RD/"
RF = "castle/test_gen_RF/"
RL = "castle/test_gen_RL/"
RR = "castle/test_gen_RR/"
RU = "castle/test_gen_RU/"


folder = R_F_D


img_source_path = folder + "test.rgba.00000.png"
img_depth_path = folder + "test.depth.00000.png"
img_goal_path = folder + "test.rgba.00005.png"


from calculate_flow import FlowNet2Utils
flow_utils = FlowNet2Utils()
img_src = read_gen(img_source_path)
img_goal = read_gen(img_goal_path)
f12 = flow_utils.flow_calculate(img_src, img_goal)
flow_utils.writeFlow(folder+"flow.flo", f12)

data = read_gen(folder+"flow.flo")
flow_img = flow_to_image(data)
img_source_disp = plt.imread(img_source_path)
img_goal_disp = plt.imread(img_goal_path)
img_depth_disp = plt.imread(img_depth_path)
d1 = plt.imread(img_goal_path)
print(folder)
plt.figure(figsize=(20,20))
columns = 3
ax1 = plt.subplot(1, columns, 1)
ax1.set_title('Source Image')
plt.imshow(img_source_disp)
ax2 = plt.subplot(1, columns, 2, label="Goal")
ax2.set_title('Goal Image')
plt.imshow(img_goal_disp)
ax3 = plt.subplot(1, columns, 3, label="Flow")
ax3.set_title('Flow')
plt.imshow(flow_img)
plt.show()

In [ ]:
import sys
from model import VisualServoingLSTM
from calculate_flow import FlowNet2Utils
from interactionmatrix import InteractionMatrix
from utils.frame_utils import read_gen, flow_to_image
import habitatenv as hs

import torch

env = hs.HabitatEnv()
flow_utils = FlowNet2Utils()
intermat = InteractionMatrix()

d1 = img_depth_disp

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

hist = []
retain_graph=True
iters = 50
mpc_steps = 15

loss_fn = torch.nn.MSELoss(size_average=False)

vs_lstm = VisualServoingLSTM(seq_len=10).cuda()

for step in range(mpc_steps):
    f12 = flow_utils.flow_calculate(img_src, img_goal)
    vel, Lsx, Lsy = intermat.getData(f12, d1)
    
    #vel = torch.tensor(np.random.rand(1,6), dtype = torch.float32)
    vel = torch.tensor(vel, dtype = torch.float32).cuda()
    Lsx = torch.tensor(Lsx, dtype = torch.float32).cuda()
    Lsy = torch.tensor(Lsy, dtype = torch.float32).cuda()
    f12 = torch.tensor(f12, dtype = torch.float32).cuda()
    # Train model
    
    optimiser = torch.optim.Adam(vs_lstm.parameters(), lr=0.1)
    print("Processing Optimization Step: %d", step)
    for cnt in range(iters):
        vs_lstm.v_interm = []
        vs_lstm.f_interm = []
          # Zero out gradient, else they will accumulate between epochs
        vs_lstm.zero_grad()
          #print("Data Loaded to GPU")

        f_hat = vs_lstm.forward(vel, Lsx, Lsy)
        loss = loss_fn(f_hat, f12)
        print("Epoch ", cnt, "MSE: ", np.sqrt(loss.item())) 
        hist.append(loss.item())
        loss.backward(retain_graph=True)
        # Update parameters
        optimiser.step()
    
    vs_lstm.v_interm = []
    vs_lstm.f_interm = []
    # Output velocity and step agent
    with torch.no_grad():
        f_hat = vs_lstm.forward(vel, Lsx, Lsy)
    
    print(vs_lstm.v_interm)
    print(vs_lstm.v_interm[0][0][0])
    # apply velocity in habitat sim and update source image
    img_src, d1 = env.example(vs_lstm.v_interm[1][0], step+1)